Part One

In [49]:
import numpy as np
np.set_printoptions(linewidth=np.inf)

In [ ]:
coords = np.genfromtxt("input08.txt", delimiter=',', dtype=int)
bc_coords = np.broadcast_to(coords, (coords.shape[0],*coords.shape))
dist_matrix = ((bc_coords - np.transpose(bc_coords, axes=(1,0,2)))**2).sum(axis=2)

_, indices = np.unique(dist_matrix, return_index=True)
####Slightly slower code checks out if counts are well-behaved
# dists, indices, counts = np.unique(dist_matrix, return_index=True, return_counts=True)
# if counts[0] == dist_matrix.shape[0] and set(counts[1:]) == {2}:
#     print("Well-behaved counts (no equal distances, zero only on diagonal)")

where_is, circuits = {}, {}
for ind in indices[1:1001]:
    #finding smallest 
    i,j = ind//dist_matrix.shape[0], ind%dist_matrix.shape[0]
    # i,j = int(i),int(j)

    #now update groups with connection i,j
    if i not in where_is:
        if j not in where_is:
            where_is[i] = where_is[j] = max(circuits.keys())+1 if circuits else 0
            circuits[where_is[i]] = {i,j}
        else:
            circuits[where_is[j]].add(i)
            where_is[i] = where_is[j]
    else:
        if j not in where_is:
            circuits[where_is[i]].add(j)
            where_is[j] = where_is[i]
        else:
            if where_is[i] != where_is[j]:
                to_del = where_is[j]
                circuits[where_is[i]].update(circuits[to_del])
                for k in circuits[to_del]: where_is[k] = where_is[i]
                del circuits[to_del]

int(np.prod(sorted([len(s) for s in circuits.values()])[-3:]))

54600

Part Two

In [ ]:
coords = np.genfromtxt("input08.txt", delimiter=',', dtype=int)
bc_coords = np.broadcast_to(coords, (coords.shape[0],*coords.shape))
dist_matrix = ((bc_coords - np.transpose(bc_coords, axes=(1,0,2)))**2).sum(axis=2)

_, indices = np.unique(dist_matrix, return_index=True)
####Slightly slower code checks out if counts are well-behaved
# dists, indices, counts = np.unique(dist_matrix, return_index=True, return_counts=True)
# if counts[0] == dist_matrix.shape[0] and set(counts[1:]) == {2}:
#     print("Well-behaved counts (no equal distances, zero only on diagonal)")

#starting circuits with single elements has no effect on time for part one
where_is, circuits = {i:i for i in range(dist_matrix.shape[0])}, {i:{i} for i in range(dist_matrix.shape[0])}
for ind in indices[1:]:
    #unraveling indices
    i,j = ind//dist_matrix.shape[0], ind%dist_matrix.shape[0]
    # i,j = int(i),int(j)

    #now update groups with connection i,j
    if where_is[i] != where_is[j]:
        old_j_circuit = where_is[j]
        circuits[where_is[i]].update(circuits[old_j_circuit])
        for k in circuits[old_j_circuit]: where_is[k] = where_is[i]
        del circuits[old_j_circuit]

        if len(circuits) == 1: break
    
    # print(len(circuits), d, end="\r") #unnecessary slowdown

print(f"Last ones connected are i={i} at {coords[i]} and j={j} at {coords[j]}, with x-product equals {int(coords[i,0]*coords[j,0])}")

Last ones connected are i=136 at [17194 12279 10275] and j=624 at [6238 3377 1071], with x-product equals 107256172


5s for part one, 38s for part two\
Reduced to 0.3s and 0.5s by not re-searching the value each time, making better use of np.unique with return_index=True (+small time-save by removing return_counts)